## <font color="#000080" size="+3">1. Setup</font>


In [65]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, tf_keras, witwidget


In [68]:
!pip show keras

Name: keras
Version: 3.3.3
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: keras-tuner, tensorflow


In [67]:
!pip install --quiet keras==3.3.3
!pip show keras

## <font color="#000080" size="+3">2. Custom layer</font>


In [83]:
import tensorflow as tf
from tensorflow.keras import layers, backend as K

class PAM(layers.Layer):
    """
    Position Attention Module (PAM) as a custom Keras Layer.

    This layer captures spatial attention mechanisms within a feature map.
    It uses three convolutional layers to project the input into different
    spaces to calculate attention, then scales the input feature map by the
    attention scores to enhance features with inter-spatial relevance.

    Attributes:
        gamma_initializer (str or keras.initializers): Initializer for the scaling factor.
        gamma_regularizer (str or keras.regularizers): Regularizer for the scaling factor.
        gamma_constraint (str or keras.constraints): Constraint for the scaling factor.
        activation (str): The type of activation function to use ('softmax' or 'sigmoid').
    """
    def __init__(self, gamma_initializer='zeros', gamma_regularizer=None, gamma_constraint=None, activation='sigmoid', **kwargs):
        super(PAM, self).__init__(**kwargs)
        self.gamma_initializer = gamma_initializer
        self.gamma_regularizer = gamma_regularizer
        self.gamma_constraint = gamma_constraint
        self.activation = activation

    def build(self, input_shape):
        if input_shape[-1] is None:
            raise ValueError("The channel dimension of the inputs should be defined. Found `None`.")

        self.gamma = self.add_weight(
            name='gamma', 
            shape=(1,),
            initializer=self.gamma_initializer,
            regularizer=self.gamma_regularizer,
            constraint=self.gamma_constraint
        )

        num_channels = input_shape[-1]
        self.conv_b = layers.Conv2D(num_channels // 8, 1, use_bias=False, kernel_initializer='he_normal')
        self.conv_c = layers.Conv2D(num_channels // 8, 1, use_bias=False, kernel_initializer='he_normal')
        self.conv_d = layers.Conv2D(num_channels, 1, use_bias=False, kernel_initializer='he_normal')
    
        # Build the internal Conv2D layers with the correct input shape
        self.conv_b.build(input_shape)
        self.conv_c.build(input_shape)
        self.conv_d.build(input_shape)
        
        super(PAM, self).build(input_shape)

    def call(self, inputs):
        shape = tf.shape(inputs)
        batch_size, h, w, filters = shape[0], shape[1], shape[2], shape[3]


        # Convolution layers to transform the input into two feature spaces B and C
        b = self.conv_b(inputs)
        c = self.conv_c(inputs)
        # Convolution layer to transform the input for output scaling
        d = self.conv_d(inputs)

        # Reshape B and C for matrix multiplication
        vec_b = K.reshape(b, (batch_size, h * w, filters // 8))
        vec_cT = K.permute_dimensions(K.reshape(c, (batch_size, h * w, filters // 8)), (0, 2, 1))

        # Attention map created by matrix multiplying B and C^T
        bcT = K.batch_dot(vec_b, vec_cT)
        
        # Apply activation to attention scores
        if self.activation == 'softmax':
            attention_scores = layers.Softmax(axis=-1)(bcT)
        elif self.activation == 'sigmoid':
            attention_scores = layers.Activation('sigmoid')(bcT)
        else:
            raise ValueError("Unsupported activation function. Choose 'softmax' or 'sigmoid'.")

        # Scale output with attention scores
        vec_d = K.reshape(d, (batch_size, h * w, filters))
        bcTd = K.batch_dot(attention_scores, vec_d)
        bcTd = K.reshape(bcTd, (batch_size, h, w, filters))

        # Apply the learnable parameter gamma to scale the attended output
        out = self.gamma * bcTd + inputs
        return out

    def compute_output_shape(self, input_shape):
        return input_shape


In [81]:
from keras.models import Model
from keras.layers import Input
import keras

# Define the input tensor
input_tensor = Input(shape=(None, None, 1408))  # Replace 'channels' with the actual number

# Create an instance of your custom layer
attention_layer = PAM()

# Apply your custom layer
output_tensor = attention_layer(input_tensor)

# Build the model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Summary of the model to see all parameters including those from PAM
model.summary()


Model: "functional_60"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_27 (InputLayer)     │ (None, None, None,     │             0 │
│                                 │ 1408)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pam_44 (PAM)                    │ (None, None, None,     │     2,478,081 │
│                                 │ 1408)                  │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,478,081 (9.45 MB)

 Trainable params: 2,478,081 (9.45 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Total params: 2,478,081 (9.45 MB)
#  Trainable params: 2,478,081 (9.45 MB)
#  Non-trainable params: 0 (0.00 B)

## <font color="#000080" size="+3">3. Unit testing</font>


### <font color="amber" size="+2">3.1 Shape testing</font>

In [84]:
import tensorflow as tf
from keras import layers

def test_pam_shape(input_shape):
    """
    Tests whether the PAM layer retains the same input and output shape.
    
    Args:
    input_shape (tuple): The shape of the input tensor to test, excluding batch size.
    
    Returns:
    bool: True if the shape test passes, False otherwise.
    """
    # Create a random tensor with the specified shape
    input_tensor = tf.random.normal([1] + list(input_shape))

    # Initialize the PAM layer
    pam_layer = PAM()

    # Get the output from the PAM layer
    output_tensor = pam_layer(input_tensor)

    # Check if the output shape matches the input shape
    if input_tensor.shape == output_tensor.shape:
        print("Shape Test Passed: Input shape and output shape are the same.")
        return True
    else:
        print("Shape Test Failed: Output shape does not match input shape.")
        return False

# Example usage
test_pam_shape((128, 128, 32))

Shape Test Passed: Input shape and output shape are the same.


True

### <font color="amber" size="+2">3.2 Functionality testing</font>

In [85]:
import tensorflow as tf

def create_patterned_input(input_shape, pattern_size=(2, 2)):
    """
    Creates a test input tensor with zeros and a specific pattern in the center.
    
    Args:
    input_shape (tuple): The shape of the input tensor, excluding batch size.
    pattern_size (tuple): The size of the square pattern to be placed in the center.
    
    Returns:
    tf.Tensor: The created input tensor with a pattern.
    """
    # Full zeros tensor
    base_input = tf.zeros([1] + list(input_shape), dtype=tf.float32)
    
    # Calculate the start indices for the pattern
    start_idx_h = input_shape[0] // 2 - pattern_size[0] // 2
    start_idx_w = input_shape[1] // 2 - pattern_size[1] // 2

    # Create indices for updates
    indices = []
    updates = []
    for i in range(pattern_size[0]):
        for j in range(pattern_size[1]):
            for k in range(input_shape[2]):
                indices.append([0, start_idx_h + i, start_idx_w + j, k])
                updates.append(1.0)

    # Convert lists to tensors
    indices = tf.constant(indices, dtype=tf.int32)
    updates = tf.constant(updates, dtype=tf.float32)

    # Update the base input tensor with a pattern
    pattern_input = tf.tensor_scatter_nd_update(base_input, indices, updates)
    return pattern_input

def test_pam_functionality(input_shape):
    """
    Tests the functionality of the PAM layer to ensure it alters the input tensor.
    
    Args:
    input_shape (tuple): The shape of the input tensor to test, excluding batch size.
    
    Returns:
    bool: True if the functionality test passes, False otherwise.
    """
    # Create a controlled test input with a specific pattern
    test_input = create_patterned_input(input_shape, pattern_size=(1, 1))

    # Initialize the PAM layer
    pam_layer = PAM(gamma_initializer='ones')

    # Get the output from the PAM layer
    output_tensor = pam_layer(test_input)

    # Calculate mean of the input and output tensors
    input_mean = tf.reduce_mean(test_input)
    output_mean = tf.reduce_mean(output_tensor)

    # Check if there is a statistical difference between the input and output
    if not tf.math.equal(input_mean, output_mean):
        print("Functionality Test Passed: Output is statistically different from the input.")
        return True
    else:
        print("Functionality Test Failed: Output is statistically the same as the input.")
        return False

# Example usage
test_pam_functionality((2, 20, 1408))


Functionality Test Passed: Output is statistically different from the input.


True